In [1]:
import gentrl
import torch
# torch.cuda.set_device(0)

In [2]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
# model.cuda();

In [3]:
model.load('saved_gentrl/')
# model.cuda();

In [4]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol

from moses.utils import disable_rdkit_log
disable_rdkit_log()

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [5]:
# num_iterations = 100000
# Kernel keeps dying after too many iterations, dunno why
num_iterations = 100
model.train_as_rl(penalized_logP, num_iterations=num_iterations)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.212;valid_perc: 0.1798;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!mean_reward: -4.161;valid_perc: 0.1879;
!

In [6]:
! mkdir -p saved_gentrl_after_rl

OSError: [Errno 12] Cannot allocate memory

In [ ]:
model.save('./saved_gentrl_after_rl/')